In [1]:
import torch
import numpy as np

import heapq

import matplotlib.pyplot as plt
import os
import copy
import time

import cv2

from receptive_field import compute_rf_prototype
from helpers import makedir, find_high_activation_crop
import shutil

import torch.utils.data
import pandas as pd
# import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import argparse
import re

from helpers import makedir
import model
import push
import prune
import train_and_test as tnt
import save
from log import create_logger
from preprocess import mean, std, preprocess_input_function

from bounding_box_metrics import bounding_box_overlap
from find_nearest import find_k_nearest_patches_to_prototypes, imsave_with_bbox, ImagePatch, ImagePatchInfo

from settings import train_dir, test_dir, train_push_dir, train_batch_size, test_batch_size, train_push_batch_size
from settings import base_architecture, img_size, prototype_shape, num_classes, prototype_activation_function, add_on_layers_type

In [2]:
ppnet = torch.load(r'/scratch/users/jiaxun1218/saved_models/vgg19_10/r1_0.7380.pth')
#ppnet_multi = ppnet.cuda()
ppnet_multi = torch.nn.DataParallel(ppnet).cuda()

In [3]:
#train_push_dir = './stanford_cars/car_data/car_data/train'
#test_dir = './stanford_cars/car_data/car_data/test'
#train_dir = './stanford_cars/car_data/car_data/train_augmented'
normalize = transforms.Normalize(mean=mean, std=std)

train_dataset = datasets.ImageFolder(
        train_push_dir,
        transforms.Compose([
        transforms.Resize(size=(img_size, img_size)),
        transforms.ToTensor(),
        normalize,
    ]))
train_push_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=train_batch_size, shuffle=True,
    num_workers=1, pin_memory=False)

In [4]:
train_dataset

Dataset ImageFolder
    Number of datapoints: 5995
    Root location: /scratch/users/jiaxun1218/data/train_cropped/
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
               Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
           )

In [5]:
find_k_nearest_patches_to_prototypes(train_push_loader, # pytorch dataloader (must be unnormalized in [0,1])
                                         ppnet_multi, # pytorch network with prototype_vectors
                                         k=1,
                                         preprocess_input_function=None, # normalize if needed
                                         full_save=True, # save all the images
                                         root_dir_for_saving_images='./icml_figures/R1_vgg_bird/',
                                         log=print,
                                         prototype_activation_function_in_numpy=None)

find nearest patches
1.0
batch 0
batch 1
batch 2
batch 3
batch 4
batch 5
batch 6
batch 7
batch 8
batch 9
batch 10
batch 11
batch 12
batch 13
batch 14
batch 15
batch 16
batch 17
batch 18
batch 19
batch 20
batch 21
batch 22
batch 23
batch 24
batch 25
batch 26
batch 27
batch 28
batch 29
batch 30
batch 31
batch 32
batch 33
batch 34
batch 35
batch 36
batch 37
batch 38
batch 39
batch 40
batch 41
batch 42
batch 43
batch 44
batch 45
batch 46
batch 47
batch 48
batch 49
batch 50
batch 51
batch 52
batch 53
batch 54
batch 55
batch 56
batch 57
batch 58
batch 59
batch 60
batch 61
batch 62
batch 63
batch 64
batch 65
batch 66
batch 67
batch 68
batch 69
batch 70
batch 71
batch 72
batch 73
batch 74


ValueError: Floating point image RGB values must be in the 0..1 range.